In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from src.utils.general import get_db_conn, get_boto3_session, load_model_s3, load_matrix_s3
from src.utils import project_constants as constants

In [3]:
cred_file = '../conf/local/credentials.yaml'
db_conn=get_db_conn(cred_file)

**Goal**

The overall goal of this notebook is to comeup with a strategy to clean up old triage experiments withtout disrupting currently used ones. 

The biggest space-hog are the `test_results.predictions` and `train_predictions.predictions` tables. If we can identify the `model_ids` 


#### Experiment hashes to remove

We can remove details about experiments based on a reference date, or based on model comments.

**Using a model comment**

In [4]:
# comm = 'initial_model_bill_passage'

# q = """
#     select 
#         experiment_hash
#     from triage_metadata.experiments
#     where config ->> 'model_comment' = '{comment_to_remove}'
# """.format(
#     comment_to_remove=comm
# )

# exp_hashes_to_remove = pd.read_sql(q, db_conn)['experiment_hash'].tolist()

**Using a reference date**

In [14]:
ref_date = '2020-12-01'
q = """
    select 
        distinct experiment_hash 
    from triage_metadata.experiment_runs where start_time < '{}';

""".format(ref_date)

exp_hashes_to_remove = pd.read_sql(q, db_conn)['experiment_hash'].tolist()

In [15]:
print(exp_hashes_to_remove)

['0e0da3905b34ab79a64b953673fd3d44', 'f454b49de52e651092235dcca538a627', '86de53310a3a08734b4a2ea5f3187aca', '8698e10f76c5e49ebc677f3063673646', '9b8574981640a3437dfaec643033d21d', '4b764fd0f77baf29bfcfa063f13b4683', '22a1d6f553479cb800205dcbcd364114', 'ba00513377f62d760ad3683215c15562', '34f048b42874e7abe23a0d87af9e7e95', '2ad4c1c4c1abd01ca5d84d57858c6a5d', '04ba097c1eeb4efd256953d339e22f1a', 'a04fee8ac7acfd268900513656260d29', 'ea790bbe28633ca9b6afd325dc3d21a9', 'd0290955199da63495f6e1c947a5b898', 'bb168764edcb5c27955514507eeb946f', 'a293552cca7f83500213ff2e397d8c61', 'cf2b49cfb1b9b4432bd59ffb3fe1bfca', '293e223b92fc2faa530204c98bf878a6', '7997e25e151e591e419efd78a899ceab', 'a8d9969b71442c1ca2188f05c422c6fe', 'e4aac155e1f12d4dd26671f5fe635820', '03105a6bbc33827324e43d524fdc3a72', '2f519b99756dd7ef7200d309985ea485', '785a3b93672acdc10da0dea563bce9d3', '59b005eef785c6b2ef27a22a240881ee', 'a0515c663d3d59c1904e5bd0c8ccc259', '6ac911beae0a5e3be2e8c4af8e1e0625', 'e6ead72c9c88fe81d3ee78f35e

**Models built by those experiments**

In [16]:
q = """
    select
        model_id, model_hash 
    from triage_metadata.models 
    where 
    built_by_experiment in {}
""".format(tuple(exp_hashes_to_remove))
           
related_models = pd.read_sql(q, db_conn)

In [17]:
# q = """
#     select
#         distinct model_hash
#     from triage_metadata.experiment_models
#     where experiment_hash in {}
# """.format(tuple(exp_hashes_to_remove))
# related_models2 = pd.read_sql(q, db_conn)

In [18]:
related_models

,model_id,model_hash
0,1,4cfaa7ce6504a563652edf627eb33666
1,2,9c22e2b58f87093e36758989b7d995a3
2,3,2d371ee1b60cf6892088c7ecbdb2460c
3,4,3b4c38fc72f9f98279596e0eacfb7915
4,5,a2e46b7774472a439da9874dfd7c6ec0
...,...,...
7357,7358,ce8aa3f4eed5444e8f0e23422a74c434
7358,7359,f09b88f4e20d35194082c45143debad8
7359,7360,7c17f70fa1052f51aee61cde08f1cdc8
7360,7361,8b1924032b2ec08cc186d4ae96755a1b


All of these models can't be removed. We need to check whether the models are used in other potentially important experiments

In [19]:
q = """
   select model_hash from triage_metadata.experiment_models where model_hash in {}
   group by model_hash having count(*) > 1
""".format(
    tuple(related_models['model_hash'].tolist())
)
models_with_multiple_exps = pd.read_sql(q, db_conn)['model_hash'].tolist()

**Checking whether the models that appear in multiple experiments are still useful**

In [20]:
q = """
    select distinct experiment_hash from triage_metadata.experiment_models where model_hash='{}'
"""

for mod in models_with_multiple_exps:
    # experiments that contain the model
    exps = pd.read_sql(q.format(mod), db_conn)['experiment_hash'].tolist()
    
    for e in exps:
        if e not in exp_hashes_to_remove:
            # removing the model from the list of models to be deleted 
            msk = related_models['model_hash'] == mod
            related_models = related_models[~msk]

**Matrices**

In [34]:
q = """
    select
        * 
    from triage_metadata.matrices 
    where 
    built_by_experiment in {}
""".format(tuple(exp_hashes_to_remove))
           
related_matrices = pd.read_sql(q, db_conn)

In [35]:
q = """
   select matrix_uuid from triage_metadata.experiment_matrices where matrix_uuid in {}
   group by matrix_uuid having count(*) > 1
""".format(
    tuple(related_matrices['matrix_uuid'].tolist())
)
mats_with_multiple_exps = pd.read_sql(q, db_conn)['matrix_uuid'].tolist()

In [37]:
q = """
    select distinct experiment_hash from triage_metadata.experiment_matrices where matrix_uuid='{}'
"""

for mat in mats_with_multiple_exps:
    # experiments that contain the matrix
    exps = pd.read_sql(q.format(mat), db_conn)['experiment_hash'].tolist()
    
    for e in exps:
        if e not in exp_hashes_to_remove:
            # removing the model from the list of models to be deleted 
            msk = related_matrices['matrix_uuid'] == mat
            related_matrices = related_matrices[~msk]

### Removing Predictions/ evaluations

We can remove the predictions for the model ids that are only related to the experiments we are not interested in

In [38]:
related_models.shape

(1146, 2)

In [39]:
# q = """
#     delete from test_results.predictions where model_id in {}
# """.format(
#     tuple(related_models['model_id'].tolist())
# )

# cursor = db_conn.cursor()

# cursor.execute(q)

In [40]:
# q = """
#     delete from test_results.evaluations where model_id in {}
# """.format(
#     tuple(related_models['model_id'].tolist())
# )

# cursor = db_conn.cursor()

# cursor.execute(q)
# db_conn.commit()

In [21]:
q = """
    delete from train_results.predictions where model_id in {}
""".format(
    tuple(related_models['model_id'].tolist())
)

cursor = db_conn.cursor()

cursor.execute(q)
db_conn.commit()

In [22]:
q = """
    delete from train_results.evaluations where model_id in {}
""".format(
    tuple(related_models['model_id'].tolist())
)

cursor = db_conn.cursor()

cursor.execute(q)
db_conn.commit()

#### Removing models/matrices

In [14]:
# q = """
#     delete from triage_metadata.models where model_id in {}
# """.format(
#    tuple(related_models['model_id'].tolist())
# )

# cursor = db_conn.cursor()
# cursor.execute(q)

ForeignKeyViolation: update or delete on table "models" violates foreign key constraint "feature_importances_model_id_fkey" on table "feature_importances"
DETAIL:  Key (model_id)=(1) is still referenced from table "feature_importances".
